In [10]:
import cv2
import mmcv
import os
import numpy as np
from mmcv.transforms import Compose
from mmengine.utils import track_iter_progress
from mmdet.registry import VISUALIZERS
from mmdet.apis import init_detector, inference_detector
from tqdm import tqdm

ModuleNotFoundError: No module named 'mmcv'

In [3]:
im_path = '/home/yaniv.sheinfeld/data/TEST_SET_6.6.6/201/images/20221124_082931_791875_201_ok.jpg'
def get_gt(filename):
    #if filename ends with ng then it is a positive sample
    #if filename ends with ok then it is a negative sample
    if filename.endswith('ng.jpg'):
        return 1
    elif filename.endswith('ok.jpg'):
        return 0

def get_tp_fp_tn_fn(pred, gt):
    if pred == 1 and gt == 1:
        return 1, 0, 0, 0
    elif pred == 1 and gt == 0:
        return 0, 1, 0, 0
    elif pred == 0 and gt == 0:
        return 0, 0, 1, 0
    elif pred == 0 and gt == 1:
        return 0, 0, 0, 1
    else:
        raise Exception('invalid input')
    return tp, fp, tn, fn
    

In [4]:
# Specify the path to model config and checkpoint file
config_file = 'configs/je/mask-rcnn_r50-caffe_fpn_ms-poly-1x_coco_je.py'
#checkpoint_file = 'work_dirs/mask-rcnn_r50-caffe_fpn_ms-poly-1x_coco_je/epoch_1.pth'
test_path = '/home/yaniv.sheinfeld/data/TEST_SET_6.6.6/201/images'
#build the model from a config file and a checkpoint file
#model = init_detector(config_file, checkpoint_file, device='cuda:0')

#init visualizer
#visualizer = VISUALIZERS.build(model.cfg.visualizer)

#the dataset_meta is loaded from the checkpoint file and then pass to the model in init_detector
#visualizer.dataset_meta = model.dataset_meta

#im_path = '/home/yaniv.sheinfeld/data/TEST_SET_6.6.6/201/images/20221124_082931_791875_201_ng.jpg'
#result = inference_detector(model, im_path)
thresholds = np.arange(0.1, 0.5, 0.1)
#for each threshold, calculate Detection Rate and False Positive Rate on all images in test_path
for epoch in range(1,21):
    checkpoint_file = f'work_dirs/mask-rcnn_r50-caffe_fpn_ms-poly-1x_coco_je/epoch_{epoch}.pth'
    model = init_detector(config_file, checkpoint_file, device='cuda:0')
    for threshold in thresholds:
        print(f'epoch: {epoch}, threshold: {threshold}')
        #init counters
        TP = 0
        FP = 0
        TN = 0
        FN = 0
        #for each image in test_path
        for im in tqdm(os.listdir(test_path), desc='test'):
            #inference
            im_path = os.path.join(test_path, im)
            result = inference_detector(model, im_path)
            pred = any(result.pred_instances.scores > threshold)
            #get the ground truth from the image path
            gt = get_gt(im_path)
            #calculate the number of true positives, false positives and false negatives
            tp, fp, tn, fn = get_tp_fp_tn_fn(pred, gt)
            TP += tp
            FP += fp
            TN += tn
            FN += fn
        #calculate Detection Rate and False Positive Rate
        DR = TP / (TP + FN)
        FPR = FP / (FP + TN)
        print('DR: ', DR)
        print('FPR: ', FPR)
        print('')


NameError: name 'np' is not defined

In [8]:
epoch_13_path = 'work_dirs/mask-rcnn_r50-caffe_fpn_ms-poly-1x_coco_je/epoch_13.pth'
th = 0.4
model = init_detector(config_file, checkpoint_file, device='cuda:0')
test_path = '/home/yaniv.sheinfeld/data/TEST_SET_6.6.6/101/images'
FN_folder = '/home/yaniv.sheinfeld/data/test_results/FN'
FP_folder = '/home/yaniv.sheinfeld/data/test_results/FP'

#init visualizer
visualizer = VISUALIZERS.build(model.cfg.visualizer)

#the dataset_meta is loaded from the checkpoint file and then pass to the model in init_detector
visualizer.dataset_meta = model.dataset_meta

#im_path = '/home/yaniv.sheinfeld/data/TEST_SET_6.6.6/201/images/20221124_082931_791875_201_ng.jpg'
for im in tqdm(os.listdir(test_path), desc='test'):
    #inference
    im_path = os.path.join(test_path, im)
    result = inference_detector(model, im_path)
    pred = any(result.pred_instances.scores > th)
    #get the ground truth from the image path
    gt = get_gt(im_path)
    #calculate the number of true positives, false positives and false negatives
    tp, fp, tn, fn = get_tp_fp_tn_fn(pred, gt)
    if fp:
        img = mmcv.imread(im_path)
        visualizer.add_datasample(f'{im}',img,data_sample = result,out_file = os.path.join(FP_folder, im),pred_score_thr = th)
    if fn:
        img = mmcv.imread(im_path)
        visualizer.add_datasample(f'{im}',img,data_sample = result,out_file = os.path.join(FN_folder, im),pred_score_thr = th)
    

Loads checkpoint by local backend from path: work_dirs/mask-rcnn_r50-caffe_fpn_ms-poly-1x_coco_je/epoch_13.pth
06/07 13:10:32 - mmengine - WARNING - `Visualizer` backend is not initialized because save_dir is None.


test: 100%|██████████| 926/926 [04:10<00:00,  3.70it/s]
